# 07_Live_Trading_Simulation

Demonstrate the RL agent in a simulated real-time (on-the-fly) trading environment using live/fresh market data.


In [ ]:
import time
import numpy as np
import pandas as pd
from src.environment.Multi_asset_env import MultiAsset21DeepHedgingEnv
from src.environment.option_pricing import create_synthetic_option_chain
from src.config.settings import get_config

import yfinance as yf
import matplotlib.pyplot as plt


In [ ]:
# Simulate fetching new spot/option prices with each step (demo: use last N points)
cfg = get_config('data')
df_live = yf.download(cfg['symbol'], start='2024-01-01', end=cfg['end_date'], interval='1d').reset_index()
df_live = df_live.rename(columns={'Date': 'date', 'Close': 'close'})
option_chain_live = create_synthetic_option_chain(df_live, get_config('option'))

strikes = get_config('option')['strike_offsets']
expiries = get_config('option')['expiry_days']
types_ = get_config('option')['option_types']
asset_universe = [{'strike_offset': s, 'expiry_days': e, 'type': t}
                  for e in expiries for s in strikes for t in types_]

env = MultiAsset21DeepHedgingEnv(df_live, option_chain_live, asset_universe)


In [ ]:
# Load your best model trained previously
from stable_baselines3 import PPO
model = PPO.load("results/models/ppo_polyhedge_agent", env=env)   # Adjust path if needed

obs, _ = env.reset()
rewards, cum_pnl, actions = [], [], []
total = 0.0

for _ in range(50):  # Simulate 50 live steps
    action, _ = model.predict(obs, deterministic=True)
    obs, reward, done, trunc, info = env.step(action)
    rewards.append(reward)
    total += reward
    cum_pnl.append(total)
    actions.append(action)
    if done: break
    time.sleep(0.1)  # Emulate live delay for demonstration

plt.plot(cum_pnl)
plt.title("Pseudo-Live Simulated Cumulative P&L")
plt.xlabel("Step")
plt.ylabel("Cumulative Reward")
plt.show()


## 7.2 Inspect Agent's Actions Over Time


In [ ]:
actions_arr = np.array(actions)
plt.figure(figsize=(12,5))
plt.plot(actions_arr[:, 0], label='Spot')
for i in range(1, 6):  # Plot first 5 option legs only for clarity
    plt.plot(actions_arr[:, i], alpha=0.7, label=f'Opt{i}')
plt.title("Positions (Spot + First 5 Options)")
plt.xlabel("Step")
plt.ylabel("Position size")
plt.legend()
plt.show()
